In [ ]:
import os
from utils.denoising_utils import *

FOLDER = "BSDS300/images/train/"
IMAGES = [FOLDER + img for img in os.listdir(FOLDER)][:100]
def L2(A):
    A_1D = np.reshape(A, (-1))
    return np.sqrt(A_1D.dot(A_1D) / A_1D.shape[0])
    
data = []
for sigma in np.arange(0,105,5):
    print("{}/100".format(sigma))
    stddev = []
    mean = []
    for fname in IMAGES:
        img_pil = crop_image(get_image(fname, -1)[0], d=32)
        img_np = pil_to_np(img_pil)
        _, img_noisy_np = get_noisy_image(img_np, sigma/255)
        diff_n = img_noisy_np - img_np
        
        stddev.append(diff_n.std()*255)
        mean.append(diff_n.mean()*255)
    data.append((sigma, np.mean(stddev), np.mean(mean)))


In [ ]:
data

In [ ]:
a, = plt.plot([(x[0]) for x in data], [(x[0]) for x in data], label="Theoretical unclipped method noise ", linestyle="dotted")
b, = plt.plot([(x[0]) for x in data], [(x[1]) for x in data], label="Empirical clipped method noise")
plt.legend(handles=[a, b])
plt.ylabel("Measured standard deviation of the method noise")
plt.xlabel("Standard deviation σ of the corrupting noise N")
axes = plt.gca()
# axes.set_xlim([xmin,xmax])
# axes.set_ylim([0,100])
plt.savefig(fname="method_noise_vs_corrupting_noise.pdf", format='pdf')

In [ ]:
coeffs_se = [-3.1389654e-6, -7.0210049e-1, -1.8598057, 3.9881199, -8.3760888, 9.7330082, -6.9596693, 2.9464712, -7.3358565e-1, 9.9630886e-2, -5.7155088e-3]
coeffs_sm = [1.6923658e-1, 7.0309281e-1, 3.7234715e-2, 2.4377832e-2, 2.0282884e-3, -1.7851033e-5, -8.5123452e-5, -2.6295693e-5, 3.2172868e-7, 1.4308530e-6]

In [ ]:
import torch
dtype = torch.cuda.FloatTensor
# p = np.polynomial.Polynomial(coeffs)
def p_se(x):
    return sum([(x**p)*coeff for p, coeff in enumerate(coeffs_se)])

def p_sm(x):
    return sum([(x**p)*coeff for p, coeff in enumerate(coeffs_sm)])

def Se(ksi):
    if not torch.is_tensor(ksi):
        ksi = dtype([ksi])
    return 1 - torch.exp(p_se(torch.sqrt(ksi)))

def Sm(mu):
    if not torch.is_tensor(mu):
        mu = dtype([mu])
    return (1+torch.tanh(p_sm(mu)))/2

def clipped_var(image, variance):
    return variance*Se(image/variance)*Se((1-image)/variance)# + 0.5*variance*Se(image/variance)*Se((1-image)/variance)

def rms(values):
    return torch.sqrt((torch.sum(values*values))/values.nelement())

def average_clipped_var(image, variance):
    image = torch.from_numpy(image).type(dtype)
#     return torch.sqrt(torch.sum(clipped_var(image, variance)**2)/image.nelement())
    return torch.mean(clipped_var(image, variance))

In [ ]:
clipped_var(1.0, 100/255)*255

In [ ]:
n = 1000000
sigma = 50
v = np.full(n, 1.0)
noise = np.random.normal(loc=np.full(n, 0), scale=sigma/255)
clipped = np.clip(v + noise, 0.0, 1.0)
# print(255*np.abs(-v + clipped))
print(255*L2(-v + clipped))
print(255*average_clipped_var(clipped, sigma/255))
# print(clipped)
# print(noise)

In [ ]:
stddev = []
mean = []
sigma = 100
for fname in IMAGES:
    img_pil = crop_image(get_image(fname, -1)[0], d=32)
    img_np = pil_to_np(img_pil)
    _, img_noisy_np = get_noisy_image(img_np, sigma/255, clip=True)
    diff_n = img_noisy_np - img_np

    
    computed = diff_n.std()*255
#     expected = computed
    expected = 255*average_clipped_var(img_noisy_np, sigma/255)
    print("Computed:\t{:.2f}\tPredicted:\t{:.2f}\tDelta:  {:.2f}\tvar:\t{:.2f}\tmean:\t{:.2f}".format(
        computed, expected, computed-expected, img_noisy_np.std(), img_noisy_np.mean()
        
    ))


In [ ]:
np.full(200, 0.5)

In [ ]:
sigmas = np.arange(0,105,5)
mus = np.linspace(0.0, 1.0, num=101)
out = np.empty((sigmas.shape[0], mus.shape[0]), dtype=np.float)

for i, sigma in enumerate(sigmas):
    print("Sigma = {}".format(sigma))
    for j, mu in enumerate(mus):
        n = 1000000
        v = np.full(n, mu)
        noise = np.random.normal(loc=np.full(n, 0), scale=sigma/255)
        clipped = np.clip(v + noise, 0.0, 1.0)
        out[i][j] = np.std(-v + clipped)



In [ ]:
for i, sigma in enumerate(sigmas):
    print("Sigma = {}".format(sigma))
    n = 10000000
    v = np.random.uniform(size=n)
    noise = np.random.normal(loc=np.full(n, 0), scale=sigma/255)
    clipped = np.clip(v + noise, 0.0, 1.0)
    for j in range(0, 255):
        cur_clipped = clipped[np.logical_and(clipped < j+1/100, j/100 < clipped)]
        cur_v = v[np.logical_and(clipped < j+1/100, j/100 < clipped)]
        out[i][j] = np.std(-cur_v + cur_clipped)

In [ ]:
255*out[10][0]

In [ ]:
plt.plot([255*out[x][0] for x in range(0,21)])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')
x = [c*5 for c in range(out.shape[0])]
y = range(out.shape[1])

X, Y = np.meshgrid(y, x)
ax.plot_surface(X=X, Y=Y, Z=out, rstride=1, cstride=1, cmap=plt.cm.ocean)


In [ ]:
table = torch.from_numpy(out)
# torch.save(table, "sigma_mu_to_std.tensor")
if 'table' not in locals():
    table = torch.load("sigma_mu_to_std.tensor")
g_table = table.cuda()

dtype = torch.cuda.FloatTensor

def pred_std(image, sigma):
    image = torch.from_numpy(image).type(dtype)
    pred = 0.0
    for channel in range(image.shape[0]):
        for x in range(image.shape[1]):
            for y in range(image.shape[2]):
                pred += g_table[int(sigma/5)][int(100*image[channel][x][y])]**2
    return np.sqrt(pred/image.nelement())

        

In [ ]:
stddev = []
mean = []
sigma = 100
for fname in IMAGES:
    img_pil = crop_image(get_image(fname, -1)[0], d=32)
    img_np = pil_to_np(img_pil)
    _, img_noisy_np = get_noisy_image(img_np, sigma/255)
    diff_n = img_noisy_np - img_np
    computed = diff_n.std()*255
    expected = 255*pred_std(img_noisy_np, sigma)
    print("Computed:\t{:.2f}\tPredicted:\t{:.2f}\tDelta:  {:.2f}\tvar:\t{:.2f}\tmean:\t{:.2f}".format(
        computed, expected, computed-expected, img_noisy_np.std(), img_noisy_np.mean()
        
    ))


In [ ]:
sigmas = np.arange(0,105,5)
mus = np.arange(0,256,1)
out = np.empty((sigmas.shape[0], mus.shape[0]), dtype=np.float)

for i, sigma in enumerate(sigmas):
# for i, sigma in enumerate([50]):
    print("Sigma = {}".format(sigma))
    n = 10000000
    v = np.random.uniform(size=n)
    noise = np.random.normal(loc=np.full(n, 0), scale=sigma/255)
    unclipped = v+noise
    clipped = np.clip(unclipped, 0.0, 1.0)
#     plt.scatter(v, clipped)
    v *= 256
    v = v.astype(int, copy=False)
    clipped *= 256
    clipped = clipped.astype(int, copy=False)
    D = np.abs(clipped - v)
    
    for j in range(0, 256):
        out[i][j] = D[v == j].mean()
plt.plot(out[0])

In [ ]:
table = torch.from_numpy(out)
# torch.save(table, "sigma_mu_to_std.tensor")
if 'table' not in locals():
    table = torch.load("sigma_mu_to_std.tensor")
g_table = table.cuda()

dtype = torch.cuda.FloatTensor

def pred_std(image, sigma):
    image = torch.from_numpy(image).type(dtype)
    pred = 0.0
    for channel in range(image.shape[0]):
        for x in range(image.shape[1]):
            for y in range(image.shape[2]):
                pred += (g_table[int(sigma/5)][int(255*image[channel][x][y])])**2
    return np.sqrt(pred/image.nelement())

plt.plot(out[20])  

In [ ]:
stddev = []
mean = []
sigma = 100
for fname in IMAGES:
    img_pil = crop_image(get_image(fname, -1)[0], d=32)
    img_np = pil_to_np(img_pil)
    _, img_noisy_np = get_noisy_image(img_np, sigma/255)
    diff_n = img_noisy_np - img_np
    computed = diff_n.std()*255
    expected = pred_std(img_noisy_np, sigma)
    print("Computed:\t{:.2f}\tPredicted:\t{:.2f}\tDelta:  {:.2f}\tvar:\t{:.2f}\tmean:\t{:.2f}".format(
        computed, expected, computed-expected, img_noisy_np.std(), img_noisy_np.mean()
        
    ))


In [ ]:
np.sqrt(255)